In [1]:
import os

In [2]:
%pwd

'c:\\Users\\ssd\\Desktop\\PRO\\End-to-End-MLOps-Developer-Salary-Predictions\\explore'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\ssd\\Desktop\\PRO\\End-to-End-MLOps-Developer-Salary-Predictions'

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    model_name: str
    preprocessor: str
    depth: int
    l2_leaf_reg: int
    learning_rate: float
    verbose: bool
    target_column: str

In [6]:
from mlProject.utils import *

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.CatBoostRegressor
        schema =  self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            train_data_path = config.train_data_path,
            test_data_path = config.test_data_path,
            model_name = config.model_name,
            preprocessor = config.preprocessor,
            depth= params.depth,
            l2_leaf_reg= params.l2_leaf_reg,
            learning_rate= params.learning_rate,
            verbose= params.verbose,
            target_column = schema.name
            
        )

        return model_trainer_config

In [8]:
import pandas as pd
import os
from mlProject.custom_logger import logger
from catboost import CatBoostRegressor
import joblib
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

In [9]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config

    
    def train(self):
        train_data = pd.read_csv(self.config.train_data_path)
        test_data = pd.read_csv(self.config.test_data_path)


        train_x = train_data.drop([self.config.target_column], axis=1)
        test_x = test_data.drop([self.config.target_column], axis=1)
        train_y = train_data[[self.config.target_column]]
        test_y = test_data[[self.config.target_column]]


        categorical_transformer = Pipeline([('one_hot_encode', OneHotEncoder(handle_unknown='ignore', sparse_output=False))])
        original_columns = ['Country', 'Age', 'EdLevel', 'RemoteWork', 'DevType', 'OrgSize', 'YearsCodePro']
        preprocessor = ColumnTransformer(transformers=[('categorical', categorical_transformer, original_columns)],remainder='passthrough')
        data_processing_pipeline = Pipeline([('preprocessor', preprocessor)])
        
        train_x = data_processing_pipeline.fit_transform(train_x)
        test_x = data_processing_pipeline.transform(test_x)

        model = CatBoostRegressor(depth=self.config.depth, l2_leaf_reg=self.config.l2_leaf_reg,
                                  learning_rate=self.config.learning_rate, verbose=self.config.verbose
                                )
        model.fit(train_x, train_y)

        joblib.dump(model, os.path.join(self.config.root_dir, self.config.model_name))
        joblib.dump(data_processing_pipeline, os.path.join(self.config.root_dir, self.config.preprocessor))



In [10]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer_config = ModelTrainer(config=model_trainer_config)
    model_trainer_config.train()
except Exception as e:
    raise e

[2023-12-23 19:54:55,159: INFO: utils: yaml file: config\config.yaml loaded successfully]
[2023-12-23 19:54:55,166: INFO: utils: yaml file: config\params.yaml loaded successfully]
[2023-12-23 19:54:55,171: INFO: utils: yaml file: config\schema.yaml loaded successfully]
[2023-12-23 19:54:55,177: INFO: utils: created directory at: artifacts]
[2023-12-23 19:54:55,179: INFO: utils: created directory at: artifacts/model_trainer]
